In [25]:
import os
import json
import pandas as pd
import openai
from tqdm.auto import tqdm

In [26]:
openai.api_key = "" # API KEY hidden

In [80]:
df = pd.read_csv("perplexity_rankings.csv", encoding="utf-8")

In [81]:
df

,user_generated,BART_gen,deepseek_gen,QWEN_generated,t5_gen
0,Generate an image of a futuristic billboard wi...,a futuristic billboard featuring a race car pa...,"A car racing on a road, full of rabbits on the...",Generate an image of a futuristic billboard wi...,a futuristic billboard with an ad featuring a ...
1,Generate an image of a series of three charact...,a series of three character portraits of a wom...,Generate an image of a series of three charact...,Generate an image of a series of three charact...,a series of three character portraits of a wom...
2,"Generate an image of a bustling, dimly-lit mar...","a bustling, dimly lit workshop in the style of...","Generate an image of a bustling, dimly-lit mar...","Generate an image of a bustling, dimly-lit mar...",lora:Atlas:1>lora:Insane Detail Slider:0.25> l...
3,Generate an image of a futuristic robotic figu...,a futuristic robotic body made entirely of tit...,"a cyberpunk futuristic robot, full body, male,...",Generate an image of a futuristic robotic figu...,a futuristic futuristic robot robot with a met...
4,Generate an image of a woman with her back tur...,"a futuristic sci-fi city beautiful landscape,...",a masterpiece portrait of a stunning dragon gi...,Generate an image of a woman with her back tur...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
...,...,...,...,...,...
195,"Generate an image of a colossal, mythical sea ...","a colossal, serpentine sea creature with glowi...",A watercolor in the style of istvan sandorfi a...,"Generate an image of a colossal, mythical sea ...","a colossal, mythical creature with glowing blu..."
196,Generate an image of a mysterious landscape fe...,"a lone figure walking along a rocky path, surr...","serpent, pink sky, masterpiece, 8k, stardust, ...",Generate an image of a mysterious landscape fe...,a surreal fantasy landscape featuring a lone f...
197,Generate an image of displays a 3D model of a ...,displays a 3D model of a woman with a grid pat...,"A character with short white hair, wearing a g...",Generate an image of displays a 3D model of a ...,a 3d render of a character with a grid pattern...
198,"Generate an image of a futuristic, steampunk-i...","masterpiece, best quality, amazing quality, ve...","Generate an image of a futuristic, steampunk-i...","Generate an image of a futuristic, steampunk-i...",lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...


Good prompts from Civitai:

In [77]:
standard_prompts = [
    "muscle car, powerful and brutal, high velocity, streaks of light, dust, inverted background, (flat lighting, flat color, flat design, no lineart:1), spot color, triadic colors, film grain, recursion, fractals, neon, pearlescent, masterpiece, best quality, amazing quality, very aesthetic, absurdres, newest, SP-Illu",
]

In [100]:
standard_prompts = ["1girl, solo, lazypos, anime-style digital drawing, CG, side view, looking at viewer, detailed background, intricate scenery, cinematic lighting, soft pastel colors, detailed and delicate, whimsical and dreamy, soft shading, detailed textures, gentle and innocent expression, intricate and ornate, elegant and charming, <lora:Smooth_Booster_v3:0.7> <lora:TRT(Illust)0.1v:0.5> <lora:PHM_style_IL_v3.3:0.5> <lora:kaelakovalskia20IllustriousXL:0.5> kaela20, medium breasts, blonde hair, red eyes, half updo, long hair, grin, flannel skirt, pleated white and blue skirt, white thighhighs,sleeves past wrists,hair bow,long sleeves,beige blouse,,red bow, heart hair ornament, heart hair ornament, zettai ryouiki, ,white sailor collar,white frilled skirt, shoes, <lora:Classroom-IL:0.7> cl4ssrm, indoors, classroom, school desks sitting, sitting on chair, hands resting on cheek, elbow on desk, window, open window"
]

Ranking by LLM:

In [101]:
def rank_prompts(prompts: dict, model: str = "gpt-4", example_prompt: str = None) -> list[str]:
    # good prompts:
    fewshot = ""
    if example_prompt:
        fewshot = (
            "Here is one example of a high-quality Stable Diffusion prompt:\n"
            f"  \"{example_prompt}\"\n\n"
        )

    system_msg = (
        "You are an unbiased expert in crafting prompts for Stable Diffusion.\n\n"
        f"{fewshot}"
        "Do *not* favor any prompt based on its label or input order. "
        "Evaluate each candidate *solely* on:\n"
        "  1. Clarity of instruction (how unambiguous it is)\n"
        "  2. Specificity of detail (level of descriptive precision)\n"
        "  3. Creativity (novelty or vividness of imagery)\n"
        "  4. Structural similarity to the gold prompt (does it follow the same ordering of modifiers, comma-separated descriptors, and parenthetical weightings)\n"
        "  5. Likelihood to produce a compelling, high-quality image\n\n"
        "Respond *only* with a JSON object structured as:\n"
        "{\n"
        '  "ranked_prompts": [\n'
        '    {"label": "<label1>", "rank": 1},\n'
        '    {"label": "<label2>", "rank": 2},\n'
        "    ...\n"
        "  ]\n"
        "}\n"
        "Use rank=1 for the best prompt."
    )

    user_msg = json.dumps(prompts, ensure_ascii=False)
    resp = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ],
        temperature=0.0,
        max_tokens=250,
    )

    raw = resp.choices[0].message.content.strip()
    # debug: 
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        # fallback: pull first {...}
        m = re.search(r"\{.*\}", raw, flags=re.S)
        if not m:
            raise ValueError(f"Invalid JSON:\n{raw}")
        data = json.loads(m.group())

    ranked = sorted(data["ranked_prompts"], key=lambda x: x["rank"])
    return [item["label"] for item in ranked]

In [102]:
df.head(20)

,A,B,C,D,E
0,Generate an image of a futuristic billboard wi...,a futuristic billboard featuring a race car pa...,"A car racing on a road, full of rabbits on the...",Generate an image of a futuristic billboard wi...,a futuristic billboard with an ad featuring a ...
1,Generate an image of a series of three charact...,a series of three character portraits of a wom...,Generate an image of a series of three charact...,Generate an image of a series of three charact...,a series of three character portraits of a wom...
2,"Generate an image of a bustling, dimly-lit mar...","a bustling, dimly lit workshop in the style of...","Generate an image of a bustling, dimly-lit mar...","Generate an image of a bustling, dimly-lit mar...",lora:Atlas:1>lora:Insane Detail Slider:0.25> l...
3,Generate an image of a futuristic robotic figu...,a futuristic robotic body made entirely of tit...,"a cyberpunk futuristic robot, full body, male,...",Generate an image of a futuristic robotic figu...,a futuristic futuristic robot robot with a met...
4,Generate an image of a woman with her back tur...,"a futuristic sci-fi city beautiful landscape,...",a masterpiece portrait of a stunning dragon gi...,Generate an image of a woman with her back tur...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
5,"Generate an image of a woman in profile, adorn...","a woman in profile, long black hair, bangs, st...","<lora:ck-shadow-circuit-IL:0.78>,\nmasterpiece...","Generate an image of a woman in profile, adorn...",lora:crow_flyway:0.5> lora:flyway_Epic-detail_...
6,Generate an image of a young woman with striki...,a young woman with striking features and dark ...,"<lora:ck-shadow-circuit-IL:0.78>,\nmasterpiece...",Generate an image of a young woman with striki...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
7,"Generate an image of a dark, fantasy character...","a dark, fantasy character with wings and flowi...","nistyle, monochrome ink painting, black ink wi...","Generate an image of a dark, fantasy character...",lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
8,Generate an image of a figure in a hat standin...,a figure in a white hat standing next to a whi...,"A picturesque, sunny day envrioment, a blizzar...",Generate an image of a figure in a hat standin...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
9,Generate an image of a striking portrait of a ...,"a digital artwork in the style of ck-mgs, nist...",a masterpiece portrait of a stunning dragon gi...,Generate an image of a striking portrait of a ...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...


In [103]:
df.columns = ["A", "B", "C", "D", "E"]
df.head(20)

,A,B,C,D,E
0,Generate an image of a futuristic billboard wi...,a futuristic billboard featuring a race car pa...,"A car racing on a road, full of rabbits on the...",Generate an image of a futuristic billboard wi...,a futuristic billboard with an ad featuring a ...
1,Generate an image of a series of three charact...,a series of three character portraits of a wom...,Generate an image of a series of three charact...,Generate an image of a series of three charact...,a series of three character portraits of a wom...
2,"Generate an image of a bustling, dimly-lit mar...","a bustling, dimly lit workshop in the style of...","Generate an image of a bustling, dimly-lit mar...","Generate an image of a bustling, dimly-lit mar...",lora:Atlas:1>lora:Insane Detail Slider:0.25> l...
3,Generate an image of a futuristic robotic figu...,a futuristic robotic body made entirely of tit...,"a cyberpunk futuristic robot, full body, male,...",Generate an image of a futuristic robotic figu...,a futuristic futuristic robot robot with a met...
4,Generate an image of a woman with her back tur...,"a futuristic sci-fi city beautiful landscape,...",a masterpiece portrait of a stunning dragon gi...,Generate an image of a woman with her back tur...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
5,"Generate an image of a woman in profile, adorn...","a woman in profile, long black hair, bangs, st...","<lora:ck-shadow-circuit-IL:0.78>,\nmasterpiece...","Generate an image of a woman in profile, adorn...",lora:crow_flyway:0.5> lora:flyway_Epic-detail_...
6,Generate an image of a young woman with striki...,a young woman with striking features and dark ...,"<lora:ck-shadow-circuit-IL:0.78>,\nmasterpiece...",Generate an image of a young woman with striki...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
7,"Generate an image of a dark, fantasy character...","a dark, fantasy character with wings and flowi...","nistyle, monochrome ink painting, black ink wi...","Generate an image of a dark, fantasy character...",lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
8,Generate an image of a figure in a hat standin...,a figure in a white hat standing next to a whi...,"A picturesque, sunny day envrioment, a blizzar...",Generate an image of a figure in a hat standin...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...
9,Generate an image of a striking portrait of a ...,"a digital artwork in the style of ck-mgs, nist...",a masterpiece portrait of a stunning dragon gi...,Generate an image of a striking portrait of a ...,lora:Hand v2:0.5> lora:Anti-Blur:1>lora:Insane...


In [1]:
for std in standard_prompts:
    model_rankings = []
    for _, row in tqdm(df.iterrows(), total=len(df),
                       desc=f"Ranking with std `{std[:30]}…`"):
        prm = {
            "A": row["A"],
            "B": row["B"],
            "C": row["C"],
            "D": row["D"],
            "E": row["E"],
        }
        ranked = rank_prompts(prm, model="gpt-4", example_prompt=std)
        model_rankings.append(ranked)

    # save to csv
    ranks_df = pd.DataFrame(
        model_rankings,
        columns=[f"rank_{i}" for i in range(1, 6)]
    )
    out = pd.concat([df, ranks_df], axis=1)
    filename = (
        std[:20]
        .replace(" ", "_")
        .replace(",", "")
        .replace("/", "_")
        + "_ranked.csv"
    )
    out.to_csv(filename, index=False)
    print("Saved:", filename)


Visualisation:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# count how often each label appears in the 1st column
counts = out['rank_1'].value_counts().sort_index()

# plot
plt.figure()
plt.bar(counts.index, counts.values)
plt.xlabel('Prompt Label')
plt.ylabel('Frequency')
plt.title('Frequency of Each Prompt Ranked #1')
plt.tight_layout()
plt.show()

In [ ]:
out

In [ ]:
# change to num rank
rank_cols = [f"rank_{i}" for i in range(1, 6)]
long = out.melt(value_vars=rank_cols, var_name="position", value_name="label")
long["rank"] = long["position"].str.extract(r"rank_(\d)").astype(int)

# avg the ranks 
avg_rank = long.groupby("label")["rank"].mean().sort_index()

# plot
plt.figure()
plt.bar(avg_rank.index, avg_rank.values)
plt.xlabel("Prompt Label")
plt.ylabel("Average Rank")
plt.title("Average Rank per Prompt Label")
plt.tight_layout()
plt.show()